# Модель предсказаний для базового шрифта

## Подготовка данных, функций и классов

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as error_metric

from lets_plot import *
from lets_plot.mapping import as_discrete
from lets_plot.bistro.qq import qq_plot
LetsPlot.setup_html()

import os; import sys; sys.path.append(os.path.join(sys.path[0], ".."))
from util import util

In [2]:
BASIC_FONT = util.Font("Lucida Grande", 14, "normal")

In [3]:
def train_test_split_by_column(df, column, train_size=None, train_frac=.75, random_state=42):
    values = df[column].drop_duplicates().sample(n=train_size, frac=train_frac, random_state=random_state).values
    return df[df[column].isin(values)], df[~df[column].isin(values)]

def dict_to_str_for_kotlin(d):
    return "mapOf({0})".format(', '.join(["'{0}' to {1}".format(k, v) for k, v in d.items()]))

In [4]:
char_widths_df = util.get_df("../data/full/char_widths.csv", "all")
char_widths_df.head()

,char_id,char,alphabet,font_family,font_size,font_face,width
0,65,A,basic_latin,Courier,9,normal,8
1,66,B,basic_latin,Courier,9,normal,8
2,67,C,basic_latin,Courier,9,normal,8
3,68,D,basic_latin,Courier,9,normal,8
4,69,E,basic_latin,Courier,9,normal,8


In [5]:
control_df = util.get_df("../data/full/control.csv", "all")
control_train_df, control_test_df = train_test_split_by_column(control_df, "text")
control_df.head()

,text,width,alphabet,font_family,font_size,font_face,symbols_count
0,Да.,24,russian,Courier,9,normal,3.0
1,Мимо.,40,russian,Courier,9,normal,5.0
2,Один.,40,russian,Courier,9,normal,5.0
3,Идея.,40,russian,Courier,9,normal,5.0
4,Изба.,40,russian,Courier,9,normal,5.0


In [6]:
control_train_df.head()

,text,width,alphabet,font_family,font_size,font_face,symbols_count
0,Да.,24,russian,Courier,9,normal,3.0
2,Один.,40,russian,Courier,9,normal,5.0
3,Идея.,40,russian,Courier,9,normal,5.0
5,Сынок.,48,russian,Courier,9,normal,6.0
6,Близко.,56,russian,Courier,9,normal,7.0


In [7]:
control_test_df.head()

,text,width,alphabet,font_family,font_size,font_face,symbols_count
1,Мимо.,40,russian,Courier,9,normal,5.0
4,Изба.,40,russian,Courier,9,normal,5.0
13,Металл.,56,russian,Courier,9,normal,7.0
14,Сходить.,64,russian,Courier,9,normal,8.0
22,Сверкать.,72,russian,Courier,9,normal,9.0


In [8]:
basic_char_widths_df = util.filter_by_font(char_widths_df, BASIC_FONT)[["alphabet", "char", "width"]]\
    .drop_duplicates(subset=["char"]).set_index("char")
basic_char_widths_df.head()

,alphabet,width
char,,
A,basic_latin,13
B,basic_latin,13
C,basic_latin,14
D,basic_latin,14
E,basic_latin,13


In [9]:
def occurrences_number(series, symbol):
    try:
        s = series.str.count(symbol).sum()
        return s - series.size if symbol in ['$', '^'] else s
    except:
        return 0

texts_s = control_train_df.text.drop_duplicates().str[0:-1]
basic_char_weights_s = pd.Series({c: occurrences_number(texts_s, c)
                                  for c in basic_char_widths_df.index}, name="weight") + 1
basic_char_weights_s.head()

A    30
B    19
C    29
D    30
E    19
Name: weight, dtype: int64

In [10]:
basic_char_data_df = pd.concat([basic_char_widths_df, basic_char_weights_s], axis="columns")
basic_char_data_df.head()

,alphabet,width,weight
A,basic_latin,13,30
B,basic_latin,13,19
C,basic_latin,14,29
D,basic_latin,14,30
E,basic_latin,13,19


In [11]:
class ClusteringModel:
    from sklearn.cluster import KMeans

    INDEX_NAME = "char"
    WIDTH_NAME = "width"
    WEIGHT_NAME = "weight"
    CLUSTER_NAME = "cluster"
    CLUSTER_WIDTH_NAME = "cluster_width"

    predictor = None
    extra_symbol_width = None

    def __init__(self,
                 calc_cluster_width=None,
                 allow_extra_symbols=True,
                 index_name=INDEX_NAME,
                 width_name=WIDTH_NAME,
                 weight_name=WEIGHT_NAME,
                 cluster_name=CLUSTER_NAME,
                 cluster_width_name=CLUSTER_WIDTH_NAME,
                 **kmean_parameters):
        self.calc_cluster_width = calc_cluster_width
        self.allow_extra_symbols = allow_extra_symbols
        self.kmean_parameters = kmean_parameters
        self.index_name = index_name
        self.width_name = width_name
        self.weight_name = weight_name
        self.cluster_name = cluster_name
        self.cluster_width_name = cluster_width_name

    def fit(self, char_data, *, admixture=None):
        if admixture is None:
            self.predictor = self._prepare_predictor(char_data, self.kmean_parameters)
        else:
            self.predictor = pd.concat([
                self._prepare_predictor(
                    char_data[char_data[admixture] == admixture_key],
                    {**self.kmean_parameters, **{"n_clusters": admixture_n_clusters}},
                    "{0}-".format(admixture_id)
                )
                for admixture_id, (admixture_key, admixture_n_clusters) \
                    in enumerate(self._calc_admixture_clusters(char_data[admixture].value_counts()).items())
            ])
        self.extra_symbol_width = (self.calc_cluster_width or self._calc_cluster_width)(self.predictor)
        return self

    def predict(self, text, name=None):
        if isinstance(text, str):
            return round(np.sum([self._predict_char_width(c) for c in text]))
        elif isinstance(text, pd.core.series.Series):
            return self._predict_for_series(text, name if (isinstance(name, str)) else None)
        elif isinstance(text, pd.core.frame.DataFrame):
            return pd.concat([
                self._predict_for_series(text[column], name[column] if (isinstance(name, dict)) else None)
                for column in text.columns
            ], axis="columns")
        else:
            raise Exception("Bad type of input: {0}".format(type(text)))

    def _calc_admixture_clusters(self, admixture_counts):
        n_admixtures = admixture_counts.shape[0]
        n_clusters = self.kmean_parameters.get("n_clusters", n_admixtures)
        if n_clusters < n_admixtures:
            raise Exception("Too few clusters: {0} < {1}".format(n_clusters, n_admixtures))
        result = {}
        for admixture, ratio in (admixture_counts.sort_values() / admixture_counts.sum()).iteritems():
            result[admixture] = max(1, round(n_clusters * ratio))
        return result

    def _prepare_predictor(self, char_data, kmean_parameters, admixture_prefix=""):
        predictor_df = char_data[[self.width_name, self.weight_name]]
        predictor_df.index.name = self.index_name
        # Set clusters
        predictor_df[self.cluster_name] = self.KMeans(**kmean_parameters).fit(predictor_df[[self.width_name]]).labels_
        # Set cluster widths
        cluster_widths = predictor_df.groupby(self.cluster_name).apply(self.calc_cluster_width or self._calc_cluster_width)
        predictor_df[self.cluster_width_name] = predictor_df[self.cluster_name].replace(cluster_widths)
        # Sort clusters
        predictor_df.sort_values(by=self.cluster_width_name, inplace=True)
        predictor_df.cluster.replace(
            {cluster_id: "{0}{1}".format(admixture_prefix, i) \
             for i, cluster_id in enumerate(predictor_df[self.cluster_name].unique())},
            inplace=True
        )
        return predictor_df

    def _calc_cluster_width(self, r):
        return (r.width * r.weight).sum() / r.weight.sum()

    def _predict_char_width(self, c):
        try:
            return self.predictor.loc[c].cluster_width
        except KeyError as e:
            if self.allow_extra_symbols:
                return self.extra_symbol_width
            else:
                raise e

    def _predict_for_series(self, text_s, name=None):
        def split_string(s):
            return pd.Series([s[i:i+1] for i in range(len(s))])
        df = text_s.apply(split_string)
        splitted_df = df.replace(self.predictor.cluster_width).fillna(0)
        cols = splitted_df.columns
        result = splitted_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)\
                                  .fillna(self.extra_symbol_width).sum(axis=1).round().astype(int)
        result.name = name or "predict_{0}".format(text_s.name)
        return result

## Подбор параметров

In [12]:
filtered_df = util.filter_by_font(control_test_df, BASIC_FONT).copy()
data = {"n_clusters": [], "score": []}
for n_clusters in range(3, 11):
    model = ClusteringModel(calc_cluster_width=lambda r: r.width.mean(), \
                            n_clusters=n_clusters, random_state=42)
    model.fit(basic_char_data_df)
    data["n_clusters"].append(n_clusters)
    data["score"].append(error_metric(filtered_df.width, model.predict(filtered_df.text)))

ggplot(data) + \
    geom_errorbar(aes(x=as_discrete("n_clusters", order=1), ymax="score"), ymin=0) + \
    ylab("score") + \
    ggtitle("Зависимость ошибки предсказания от количества кластеров", "Без примесей")

Ошибка заметно улучшается при 7 кластерах, а дальнейшее улучшение уже не так существенно.

In [13]:
filtered_df = util.filter_by_font(control_test_df, BASIC_FONT).copy()
data = {"n_clusters": [], "score": []}
for n_clusters in range(5, 15):
    model = ClusteringModel(calc_cluster_width=lambda r: r.width.mean(), \
                            n_clusters=n_clusters, random_state=42)
    model.fit(basic_char_data_df, admixture="alphabet")
    data["n_clusters"].append(n_clusters)
    data["score"].append(error_metric(filtered_df.width, model.predict(filtered_df.text)))

ggplot(data) + \
    geom_errorbar(aes(x=as_discrete("n_clusters", order=1), ymax="score"), ymin=0) + \
    ylab("score") + \
    ggtitle("Зависимость ошибки предсказания от количества кластеров", "С примесями")

Если учитывать примеси (в рамках одного кластера не могут быть смешаны символы из разных групп, в данном случае - языковых) - то неплохим количеством кластеров должно быть 11.

## Тренировка различных моделей

In [14]:
simple_clustering_model = ClusteringModel(
    calc_cluster_width=lambda r: r.width.mean(),
    n_clusters=7,
    random_state=42
)
simple_clustering_model.fit(basic_char_data_df)
print("Простая модель")
print("Кластеры: {0}".format(dict_to_str_for_kotlin(simple_clustering_model.predictor.cluster.to_dict())))
print("Ширины кластеров: {0}".format(
    simple_clustering_model.predictor[["cluster", "cluster_width"]]\
                           .drop_duplicates().set_index("cluster").cluster_width.to_dict())
)
print("Ширина неизвестного символа: {0}".format(simple_clustering_model.extra_symbol_width))

Простая модель
Кластеры: mapOf('/' to 0, 'i' to 0, '.' to 0, 't' to 0, ' ' to 0, 'j' to 0, ']' to 0, '\' to 0, '[' to 0, 'l' to 0, ',' to 0, ''' to 0, ':' to 0, ';' to 0, '"' to 1, '-' to 1, 'f' to 1, 'r' to 1, '*' to 1, 'г' to 1, '|' to 1, '!' to 1, 'I' to 1, ')' to 1, '{' to 1, '(' to 1, '`' to 1, '}' to 1, '^' to 1, 'n' to 2, 'z' to 2, 'u' to 2, 'v' to 2, 'x' to 2, 'y' to 2, 'Г' to 2, 's' to 2, 'я' to 2, 'h' to 2, 'п' to 2, 'н' to 2, 'k' to 2, 'л' to 2, 'к' to 2, 'й' to 2, 'и' to 2, 'з' to 2, 'с' to 2, 'у' to 2, 'J' to 2, 'х' to 2, 'a' to 2, 'а' to 2, 'c' to 2, 'ч' to 2, 'э' to 2, 'в' to 2, 'т' to 2, '=' to 3, 'ц' to 3, 'ъ' to 3, 'ь' to 3, '>' to 3, '?' to 3, '_' to 3, 'о' to 3, '~' to 3, 'е' to 3, 'д' to 3, 'б' to 3, 'У' to 3, 'Б' to 3, 'Т' to 3, 'З' to 3, 'Л' to 3, 'р' to 3, 'К' to 3, '<' to 3, 'q' to 3, 'e' to 3, 'b' to 3, 'Z' to 3, 'Y' to 3, 'g' to 3, 'T' to 3, 'o' to 3, 'p' to 3, 'L' to 3, 'd' to 3, '0' to 3, '1' to 3, '+' to 3, '3' to 3, '4' to 3, '5' to 3, '6' to 3, '7' to 3,

In [15]:
weighted_clustering_model = ClusteringModel(
    n_clusters=7,
    random_state=42
)
weighted_clustering_model.fit(basic_char_data_df)
print("Модель с весами")
print("Кластеры: {0}".format(dict_to_str_for_kotlin(weighted_clustering_model.predictor.cluster.to_dict())))
print("Ширины кластеров: {0}".format(
    weighted_clustering_model.predictor[["cluster", "cluster_width"]]\
                             .drop_duplicates().set_index("cluster").cluster_width.to_dict())
)
print("Ширина неизвестного символа: {0}".format(weighted_clustering_model.extra_symbol_width))

Модель с весами
Кластеры: mapOf('/' to 0, 'i' to 0, '.' to 0, 't' to 0, ' ' to 0, 'j' to 0, ']' to 0, '\' to 0, '[' to 0, 'l' to 0, ',' to 0, ''' to 0, ':' to 0, ';' to 0, '"' to 1, '-' to 1, 'f' to 1, 'r' to 1, '*' to 1, 'г' to 1, '|' to 1, '!' to 1, 'I' to 1, ')' to 1, '{' to 1, '(' to 1, '`' to 1, '}' to 1, '^' to 1, 'n' to 2, 'z' to 2, 'u' to 2, 'v' to 2, 'x' to 2, 'y' to 2, 'Г' to 2, 's' to 2, 'я' to 2, 'h' to 2, 'п' to 2, 'н' to 2, 'k' to 2, 'л' to 2, 'к' to 2, 'й' to 2, 'и' to 2, 'з' to 2, 'с' to 2, 'у' to 2, 'J' to 2, 'х' to 2, 'a' to 2, 'а' to 2, 'c' to 2, 'ч' to 2, 'э' to 2, 'в' to 2, 'т' to 2, '=' to 3, 'ц' to 3, 'ъ' to 3, 'ь' to 3, '>' to 3, '?' to 3, '_' to 3, 'о' to 3, '~' to 3, 'е' to 3, 'д' to 3, 'б' to 3, 'У' to 3, 'Б' to 3, 'Т' to 3, 'З' to 3, 'Л' to 3, 'р' to 3, 'К' to 3, '<' to 3, 'q' to 3, 'e' to 3, 'b' to 3, 'Z' to 3, 'Y' to 3, 'g' to 3, 'T' to 3, 'o' to 3, 'p' to 3, 'L' to 3, 'd' to 3, '0' to 3, '1' to 3, '+' to 3, '3' to 3, '4' to 3, '5' to 3, '6' to 3, '7' to 3

In [16]:
admixtured_clustering_model = ClusteringModel(
    calc_cluster_width=lambda r: r.width.mean(),
    n_clusters=11,
    random_state=42
)
admixtured_clustering_model.fit(basic_char_data_df, admixture="alphabet")
print("Модель с примесями")
print("Кластеры: {0}".format(dict_to_str_for_kotlin(admixtured_clustering_model.predictor.cluster.to_dict())))
print("Ширины кластеров: {0}".format(
    admixtured_clustering_model.predictor[["cluster", "cluster_width"]]\
                               .drop_duplicates().set_index("cluster").cluster_width.to_dict())
)
print("Ширина неизвестного символа: {0}".format(admixtured_clustering_model.extra_symbol_width))

Модель с примесями
Кластеры: mapOf('я' to 0-0, 'г' to 0-0, 'с' to 0-0, 'т' to 0-0, 'у' to 0-0, 'х' to 0-0, 'н' to 0-0, 'ч' to 0-0, 'л' to 0-0, 'к' to 0-0, 'и' to 0-0, 'з' to 0-0, 'а' to 0-0, 'Г' to 0-0, 'э' to 0-0, 'в' to 0-0, 'й' to 0-0, 'п' to 0-0, 'о' to 0-1, 'е' to 0-1, 'б' to 0-1, 'Т' to 0-1, 'У' to 0-1, 'ц' to 0-1, 'Л' to 0-1, 'К' to 0-1, 'З' to 0-1, 'ъ' to 0-1, 'ь' to 0-1, 'Б' to 0-1, 'р' to 0-1, 'д' to 0-1, 'м' to 0-2, 'ш' to 0-2, 'щ' to 0-2, 'ж' to 0-2, 'ы' to 0-2, 'А' to 0-2, 'Р' to 0-2, 'В' to 0-2, 'Д' to 0-2, 'Е' to 0-2, 'И' to 0-2, 'Й' to 0-2, 'М' to 0-2, 'Н' to 0-2, 'О' to 0-2, 'П' to 0-2, 'ю' to 0-2, 'Я' to 0-2, 'Ф' to 0-2, 'Х' to 0-2, 'Ц' to 0-2, 'Ч' to 0-2, 'Ъ' to 0-2, 'Ь' to 0-2, 'Э' to 0-2, 'С' to 0-2, 'ф' to 0-3, 'Ш' to 0-3, 'Щ' to 0-3, 'Ж' to 0-3, 'Ы' to 0-3, 'Ю' to 0-3, ' ' to 1-0, 'l' to 1-0, 't' to 1-0, ''' to 1-0, 'j' to 1-0, ',' to 1-0, '.' to 1-0, '/' to 1-0, ':' to 1-0, ';' to 1-0, 'i' to 1-0, '\' to 1-0, ']' to 1-0, '[' to 1-0, 'I' to 1-1, 'f' to 1-1, '}' t

In [17]:
full_clustering_model = ClusteringModel(
    n_clusters=11,
    random_state=42
)
full_clustering_model.fit(basic_char_data_df, admixture="alphabet")
print("Модель с весами и примесями")
print("Кластеры: {0}".format(dict_to_str_for_kotlin(full_clustering_model.predictor.cluster.to_dict())))
print("Ширины кластеров: {0}".format(
    full_clustering_model.predictor[["cluster", "cluster_width"]]\
                         .drop_duplicates().set_index("cluster").cluster_width.to_dict())
)
print("Ширина неизвестного символа: {0}".format(full_clustering_model.extra_symbol_width))

Модель с весами и примесями
Кластеры: mapOf('я' to 0-0, 'г' to 0-0, 'с' to 0-0, 'т' to 0-0, 'у' to 0-0, 'х' to 0-0, 'н' to 0-0, 'ч' to 0-0, 'л' to 0-0, 'к' to 0-0, 'и' to 0-0, 'з' to 0-0, 'а' to 0-0, 'Г' to 0-0, 'э' to 0-0, 'в' to 0-0, 'й' to 0-0, 'п' to 0-0, 'о' to 0-1, 'е' to 0-1, 'б' to 0-1, 'Т' to 0-1, 'У' to 0-1, 'ц' to 0-1, 'Л' to 0-1, 'К' to 0-1, 'З' to 0-1, 'ъ' to 0-1, 'ь' to 0-1, 'Б' to 0-1, 'р' to 0-1, 'д' to 0-1, 'м' to 0-2, 'ш' to 0-2, 'щ' to 0-2, 'ж' to 0-2, 'ы' to 0-2, 'А' to 0-2, 'Р' to 0-2, 'В' to 0-2, 'Д' to 0-2, 'Е' to 0-2, 'И' to 0-2, 'Й' to 0-2, 'М' to 0-2, 'Н' to 0-2, 'О' to 0-2, 'П' to 0-2, 'ю' to 0-2, 'Я' to 0-2, 'Ф' to 0-2, 'Х' to 0-2, 'Ц' to 0-2, 'Ч' to 0-2, 'Ъ' to 0-2, 'Ь' to 0-2, 'Э' to 0-2, 'С' to 0-2, 'ф' to 0-3, 'Ш' to 0-3, 'Щ' to 0-3, 'Ж' to 0-3, 'Ы' to 0-3, 'Ю' to 0-3, ' ' to 1-0, 'l' to 1-0, 't' to 1-0, ''' to 1-0, 'j' to 1-0, ',' to 1-0, '.' to 1-0, '/' to 1-0, ':' to 1-0, ';' to 1-0, 'i' to 1-0, '\' to 1-0, ']' to 1-0, '[' to 1-0, 'I' to 1-1, 'f' to 1

## Сравнение результатов предсказаний для разных моделей

In [18]:
filtered_df = util.filter_by_font(control_test_df, BASIC_FONT).copy()
test_df = pd.concat([
    filtered_df.copy().assign(
        model=lambda x: model_name,
        width_prediction=model.predict(filtered_df.text),
    )
    for model_name, model in {
        "simple": simple_clustering_model,
        "weighted": weighted_clustering_model,
        "admixtured": admixtured_clustering_model,
        "full": full_clustering_model,
    }.items()
])
test_df["width_prediction_error"] = test_df.width_prediction - test_df.width
test_df["width_prediction_mean_error"] = test_df.width_prediction_error / test_df.symbols_count
test_df

,text,width,alphabet,symbols_count,model,width_prediction,width_prediction_error,width_prediction_mean_error
0,Мимо.,54,russian,5.0,simple,53,-1,-0.200000
1,Изба.,49,russian,5.0,simple,49,0,0.000000
2,Металл.,69,russian,7.0,simple,68,-1,-0.142857
3,Сходить.,79,russian,8.0,simple,81,2,0.250000
4,Сверкать.,88,russian,9.0,simple,91,3,0.333333
...,...,...,...,...,...,...,...,...
244,Rock seat near business loss federal growth ap...,1271,basic_latin,149.0,full,1276,5,0.033557
245,Nation production little could his under style...,1208,basic_latin,149.0,full,1227,19,0.127517
246,Throughout Mr friend six including sea strong ...,1302,basic_latin,152.0,full,1313,11,0.072368
247,Worker worker single let back three because li...,1335,basic_latin,158.0,full,1351,16,0.101266


In [31]:
test_df[test_df.model == "weighted"].width_prediction_mean_error.median()

0.021739130434782608

In [19]:
ggplot(test_df) + \
    geom_point(aes("width", "width_prediction")) + \
    geom_abline(slope=1, color="magenta", size=1) + \
    facet_grid(x="model") + \
    scale_x_log10() + scale_y_log10() + \
    ggtitle("Отношение предсказанной ширины к реальной")

Согласно графикам, ни одна из моделей не дает слишком сильных "выбросов" - предсказанная ширина всегда более или менее близка к реальной.

In [20]:
ggplot(test_df) + \
    geom_point(aes("symbols_count", "width_prediction_error")) + \
    geom_smooth(aes("symbols_count", "width_prediction_error")) + \
    facet_grid(x="model") + \
    ggtitle("Зависимость абсолютной ошибки от количества символов", \
            "Без учета языка")

In [21]:
ggplot(test_df) + \
    geom_point(aes("symbols_count", "width_prediction_error", color="alphabet")) + \
    geom_smooth(aes("symbols_count", "width_prediction_error", color="alphabet")) + \
    facet_grid(x="model") + \
    ggtitle("Зависимость абсолютной ошибки от количества символов", \
            "С учетом языка")

Если не проводить различия между языками (что и делается по умолчанию), то при увеличении длины текста лучше начинает вести себя "полная" модель - которая и учитывает веса и не позволяет языкам смешиваться в кластерах.

Если обратить внимание на то как ошибки распределены по языкам, то видно, что латиница лучше всего предсказывается "взвешенной" моделью, а кириллица - "полной". При этом "полная" модель хуже всех предсказывает латиницу, а кириллица не поддается "простой" модели.

In [22]:
ggplot(test_df) + \
    geom_area(aes(x="width_prediction_mean_error"), stat='density') + \
    facet_grid(x="model") + \
    ggtitle("Плотность распределения средней ошибки", \
            "Без учета языка")

In [23]:
ggplot(test_df) + \
    geom_density(aes(x="width_prediction_mean_error", color="alphabet"), size=2) + \
    facet_grid(x="model") + \
    ggtitle("Плотность распределения средней ошибки", \
            "С учетом языка")

Если средняя ошибка (ошибка для 1 символа) распределена нормально вокруг 0 и не слишком велика, то о ней можно сильно не беспокоиться. Если она большая или не распределена нормально - желательно улучшить модель. Если она просто немного смещена относительно 0 - ее можно скорректировать вычитанием `symbols_count * mean_error` из предсказания.

Если не делать различия между языками, то почти все модели выглядят неплохо (быть может, за исключением "полной" модели).

Если же различать языки, то наиболее выгодно смотрится "взвешенная" модель.

In [24]:
qq_plot(test_df, sample="width_prediction_mean_error") + \
    facet_grid(x="model") + \
    ggtitle("Нормальность распределения средней ошибки", \
            "Без учета языка")

In [25]:
qq_plot(test_df, sample="width_prediction_mean_error", group="alphabet") + \
    facet_grid(x="model") + \
    ggtitle("Нормальность распределения средней ошибки", \
            "С учетом языка")

Судя по Q-Q-графику, все рассматриваемые распределения ошибок более или менее могут сойти за нормальное.

## Распределение символов по кластерам

In [26]:
N_CLUSTERS_MIN, N_CLUSTERS_MAX = 3, 10
clusters_df = pd.DataFrame(columns=["char", "n_clusters", "cluster", "alphabet"])
for n_clusters in range(N_CLUSTERS_MIN, N_CLUSTERS_MAX + 1):
    model = ClusteringModel(n_clusters=n_clusters, random_state=42)
    model.fit(basic_char_data_df)
    clusters_df = pd.concat([
        clusters_df,
        pd.concat([basic_char_data_df, model.predictor.cluster], axis="columns")\
            .assign(n_clusters=lambda x: n_clusters)[["alphabet", "n_clusters", "cluster"]].reset_index(),
    ])
clusters_df.reset_index(inplace=True, drop=True)
clusters_df.cluster = clusters_df.cluster.astype(int)
clusters_df

,char,n_clusters,cluster,alphabet
0,A,3,2,basic_latin
1,B,3,2,basic_latin
2,C,3,2,basic_latin
3,D,3,2,basic_latin
4,E,3,2,basic_latin
...,...,...,...,...
1267,ы,10,7,russian
1268,ь,10,4,russian
1269,э,10,3,russian
1270,ю,10,7,russian


In [27]:
jitter_radius = .5
clusters_df = pd.merge(left=clusters_df, right=basic_char_data_df.assign(
    jitter_x=jitter_radius * np.random.uniform(size=basic_char_data_df.shape[0]) - jitter_radius / 2.0,
    jitter_y=jitter_radius * np.random.uniform(size=basic_char_data_df.shape[0]) - jitter_radius / 2.0,
)[["jitter_x", "jitter_y"]].reset_index(), on="char")
clusters_df["basic_x"] = (clusters_df.cluster - (clusters_df.n_clusters - 1) / 2.0) * (N_CLUSTERS_MAX / clusters_df.n_clusters)
clusters_df["basic_y"] = N_CLUSTERS_MAX - clusters_df.n_clusters
clusters_df["x"] = clusters_df.basic_x + clusters_df.jitter_x
clusters_df["y"] = clusters_df.basic_y + clusters_df.jitter_y
clusters_df

,char,n_clusters,cluster,alphabet,jitter_x,jitter_y,basic_x,basic_y,x,y
0,A,3,2,basic_latin,-0.077304,0.020353,3.333333,7,3.256029,7.020353
1,A,4,2,basic_latin,-0.077304,0.020353,1.25,6,1.172696,6.020353
2,A,5,3,basic_latin,-0.077304,0.020353,2.0,5,1.922696,5.020353
3,A,6,3,basic_latin,-0.077304,0.020353,0.833333,4,0.756029,4.020353
4,A,7,4,basic_latin,-0.077304,0.020353,1.428571,3,1.351267,3.020353
...,...,...,...,...,...,...,...,...,...,...
1267,я,6,1,russian,0.178385,0.193910,-2.5,4,-2.321615,4.19391
1268,я,7,2,russian,0.178385,0.193910,-1.428571,3,-1.250186,3.19391
1269,я,8,2,russian,0.178385,0.193910,-1.875,2,-1.696615,2.19391
1270,я,9,3,russian,0.178385,0.193910,-1.111111,1,-0.932726,1.19391


In [28]:
ggplot() + \
    geom_path(aes("x", "y", group="char", color="alphabet"), data=clusters_df, alpha=.3) + \
    geom_point(aes("x", "y", group="char", color="alphabet"), data=clusters_df, size=2, \
               tooltips=layer_tooltips().line("@char")) + \
    geom_point(aes("basic_x", "basic_y"), data=clusters_df.drop_duplicates(subset=["basic_x", "basic_y"]), \
               size=30, shape=1, color="black") + \
    ggsize(1000, 700) + ggtitle("Перераспределение символов по кластерам при росте числа кластеров") + \
    theme_classic() + theme(axis='blank')